In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [6]:
week10_link = 'http://www.footballlocks.com/nfl_odds_week_10.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week10_html = requests.get(week10_link).text

soup = BeautifulSoup(week10_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[256])

<tr>
<td>11/9 8:25 ET</td>
<td>Seattle</td>
<td>-6</td>
<td>At Arizona</td>
<td>40.5</td>
<td>-$270 +$230</td>
</tr>
<tr>
<td>11/13 8:30 ET</td>
<td>At Carolina</td>
<td>-10</td>
<td>Tampa Bay</td>
<td>37</td>
<td>-$575 +$425</td>
</tr>


In [7]:
#want to 257 because it does not include last one
table = tr[51:257]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week10_df = pd.DataFrame(string_table)

week10_df.columns = ['all_data']

week10_df.head()

,all_data
0,<tr>\n<td>11/9 8:25 ET</td>\n<td>Seattle</td>\...
1,<tr>\n<td>11/12 1:00 ET</td>\n<td>Minnesota</t...
2,<tr>\n<td>11/12 1:00 ET</td>\n<td>At Chicago</...
3,<tr>\n<td>11/12 1:00 ET</td>\n<td>Pittsburgh</...
4,<tr>\n<td>11/12 1:00 ET</td>\n<td>At Jacksonvi...


In [8]:
#want to clean the date data

dates = [x.split('td')[1] for x in week10_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week10_df['date'] = cleaned_dates

In [9]:
#pull out favorites

week10_df['cleaned_favorites'] = week10_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week10_df['cleaned_underdogs'] = week10_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week10_df['cleaned_spread'] = week10_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week10_df['cleaned_over_under'] = week10_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week10_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week10_df['money_odds'] = money_odds

#input week number for each game
week10_df['week_number'] = '10'

In [10]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week10_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week10_df['year'] = year_list

In [11]:
#create a final dataframe for week 10
week10_final = week10_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week10_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,11/9 8:25 ET,2018,10,Seattle,At Arizona,-6,40.5,-$270 +$230
1,11/12 1:00 ET,2018,10,Minnesota,At Washington,-1,42.5,-$120 +$100
2,11/12 1:00 ET,2018,10,At Chicago,Green Bay,-5,37.5,-$225 +$185
3,11/12 1:00 ET,2018,10,Pittsburgh,At Indianapolis,-11,47,-$575 +$425
4,11/12 1:00 ET,2018,10,At Jacksonville,LA Chargers,-5,40.5,-$230 +$190


In [12]:
#week10_final to pickle
week10_final.to_pickle('week10_final.pkl')